In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [2]:
df=pd.read_csv('train.csv')

df['Vehicle_Age'] = df['Vehicle_Age'].astype('category')
df['Vehicle_Age'] = df['Vehicle_Age'].cat.rename_categories({'< 1 Year': 0,'1-2 Year': 1,'> 2 Years': 2})
df['Vehicle_Age'] = df['Vehicle_Age'].astype('int8')

df['Vehicle_Damage'] = df['Vehicle_Damage'].astype('category')
df['Vehicle_Damage'] = df['Vehicle_Damage'].cat.rename_categories({'No': 0,'Yes': 1,})
df['Vehicle_Damage'] = df['Vehicle_Damage'].astype('int8')

df['Gender'] = df['Gender'].astype('category')
df['Gender'] = df['Gender'].cat.rename_categories({'Female': 0,'Male': 1,})
df['Gender'] = df['Gender'].astype('int8')

print(df.info())
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11504798 entries, 0 to 11504797
Data columns (total 12 columns):
 #   Column                Dtype  
---  ------                -----  
 0   id                    int64  
 1   Gender                int8   
 2   Age                   int64  
 3   Driving_License       int64  
 4   Region_Code           float64
 5   Previously_Insured    int64  
 6   Vehicle_Age           int8   
 7   Vehicle_Damage        int8   
 8   Annual_Premium        float64
 9   Policy_Sales_Channel  float64
 10  Vintage               int64  
 11  Response              int64  
dtypes: float64(3), int64(6), int8(3)
memory usage: 822.9 MB
None


,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0,1,21,1,35.0,0,1,1,65101.0,124.0,187,0
1,1,1,43,1,28.0,0,2,1,58911.0,26.0,288,1
2,2,0,25,1,14.0,1,0,0,38043.0,152.0,254,0
3,3,0,35,1,1.0,0,1,1,2630.0,156.0,76,0
4,4,0,36,1,15.0,1,1,0,31951.0,152.0,294,0
...,...,...,...,...,...,...,...,...,...,...,...,...
11504793,11504793,1,48,1,6.0,0,1,1,27412.0,26.0,218,0
11504794,11504794,0,26,1,36.0,0,0,1,29509.0,152.0,115,1
11504795,11504795,0,29,1,32.0,1,0,0,2630.0,152.0,189,0
11504796,11504796,0,51,1,28.0,0,1,1,48443.0,26.0,274,1


In [3]:
df_0=df[df['Response']==0]
df_0

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0,1,21,1,35.0,0,1,1,65101.0,124.0,187,0
2,2,0,25,1,14.0,1,0,0,38043.0,152.0,254,0
3,3,0,35,1,1.0,0,1,1,2630.0,156.0,76,0
4,4,0,36,1,15.0,1,1,0,31951.0,152.0,294,0
5,5,0,31,1,47.0,1,0,0,28150.0,152.0,197,0
...,...,...,...,...,...,...,...,...,...,...,...,...
11504791,11504791,0,31,1,32.0,1,0,0,2630.0,152.0,204,0
11504792,11504792,1,34,1,28.0,1,1,0,29974.0,154.0,201,0
11504793,11504793,1,48,1,6.0,0,1,1,27412.0,26.0,218,0
11504795,11504795,0,29,1,32.0,1,0,0,2630.0,152.0,189,0


In [4]:
df_1=df[df['Response']==1]
df_1

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
1,1,1,43,1,28.0,0,2,1,58911.0,26.0,288,1
7,7,0,47,1,8.0,0,1,1,40659.0,26.0,262,1
28,28,1,40,1,35.0,0,1,1,2630.0,157.0,204,1
36,36,0,50,1,46.0,0,1,1,29248.0,124.0,273,1
44,44,1,40,1,8.0,0,1,1,36217.0,124.0,286,1
...,...,...,...,...,...,...,...,...,...,...,...,...
11504732,11504732,1,45,1,28.0,0,1,1,32540.0,124.0,90,1
11504755,11504755,1,45,1,36.0,0,1,1,37661.0,30.0,163,1
11504784,11504784,1,25,1,28.0,0,0,1,48240.0,153.0,37,1
11504794,11504794,0,26,1,36.0,0,0,1,29509.0,152.0,115,1


In [5]:
df_test= df_0[len(df_1)*7:]
#df_test = pd.concat((df_test,df_1[:len(df_test)]))
x_test = df_test.drop(columns=['Response','id'])
y_test= df_test['Response']
x_test

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
11294642,1,22,1,41.0,0,0,1,36440.0,152.0,216
11294643,0,24,1,8.0,1,0,0,46958.0,152.0,78
11294644,1,26,1,28.0,0,0,1,50965.0,152.0,167
11294645,0,69,1,28.0,1,1,0,40997.0,122.0,131
11294646,1,39,1,28.0,0,1,1,28752.0,124.0,81
...,...,...,...,...,...,...,...,...,...,...
11504791,0,31,1,32.0,1,0,0,2630.0,152.0,204
11504792,1,34,1,28.0,1,1,0,29974.0,154.0,201
11504793,1,48,1,6.0,0,1,1,27412.0,26.0,218
11504795,0,29,1,32.0,1,0,0,2630.0,152.0,189


In [6]:
clf_list=[]
for i in range(7):
    df_tmp = pd.concat((df_0[len(df_1)*i:len(df_1)*i+len(df_1)],df_1))
    df_tmp = df_tmp.sample(frac=1)
    
    y = df_tmp['Response']

    x = df_tmp.drop(columns=['Response','id'])

    #clf = HistGradientBoostingClassifier().fit(x, y)
    clf = HistGradientBoostingClassifier().fit(x, y)
    print(clf.score(x_test,y_test))
    
    clf_list.append(clf)
clf_list

0.6893818560593731
0.6893330295237785
0.689354730206265
0.6897344921497781
0.6906025194492367
0.689495784642427
0.6897561928322645


[HistGradientBoostingClassifier(),
 HistGradientBoostingClassifier(),
 HistGradientBoostingClassifier(),
 HistGradientBoostingClassifier(),
 HistGradientBoostingClassifier(),
 HistGradientBoostingClassifier(),
 HistGradientBoostingClassifier()]

In [7]:
df=pd.read_csv('test.csv')

df['Vehicle_Age'] = df['Vehicle_Age'].astype('category')
df['Vehicle_Age'] = df['Vehicle_Age'].cat.rename_categories({'< 1 Year': 0,'1-2 Year': 1,'> 2 Years': 2})
df['Vehicle_Age'] = df['Vehicle_Age'].astype('int8')

df['Vehicle_Damage'] = df['Vehicle_Damage'].astype('category')
df['Vehicle_Damage'] = df['Vehicle_Damage'].cat.rename_categories({'No': 0,'Yes': 1,})
df['Vehicle_Damage'] = df['Vehicle_Damage'].astype('int8')

df['Gender'] = df['Gender'].astype('category')
df['Gender'] = df['Gender'].cat.rename_categories({'Female': 0,'Male': 1,})
df['Gender'] = df['Gender'].astype('int8')

print(df.info())
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7669866 entries, 0 to 7669865
Data columns (total 11 columns):
 #   Column                Dtype  
---  ------                -----  
 0   id                    int64  
 1   Gender                int8   
 2   Age                   int64  
 3   Driving_License       int64  
 4   Region_Code           float64
 5   Previously_Insured    int64  
 6   Vehicle_Age           int8   
 7   Vehicle_Damage        int8   
 8   Annual_Premium        float64
 9   Policy_Sales_Channel  float64
 10  Vintage               int64  
dtypes: float64(3), int64(5), int8(3)
memory usage: 490.1 MB
None


,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,11504798,0,20,1,47.0,0,0,0,2630.0,160.0,228
1,11504799,1,47,1,28.0,0,1,1,37483.0,124.0,123
2,11504800,1,47,1,43.0,0,1,1,2630.0,26.0,271
3,11504801,0,22,1,47.0,1,0,0,24502.0,152.0,115
4,11504802,1,51,1,19.0,0,1,0,34115.0,124.0,148
...,...,...,...,...,...,...,...,...,...,...,...
7669861,19174659,1,57,1,28.0,0,1,1,51661.0,124.0,109
7669862,19174660,1,28,1,50.0,1,0,0,25651.0,152.0,184
7669863,19174661,1,47,1,33.0,1,1,0,2630.0,138.0,63
7669864,19174662,1,30,1,28.0,0,0,1,38866.0,124.0,119


In [8]:
x = df.drop(columns=['id'])
x

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,0,20,1,47.0,0,0,0,2630.0,160.0,228
1,1,47,1,28.0,0,1,1,37483.0,124.0,123
2,1,47,1,43.0,0,1,1,2630.0,26.0,271
3,0,22,1,47.0,1,0,0,24502.0,152.0,115
4,1,51,1,19.0,0,1,0,34115.0,124.0,148
...,...,...,...,...,...,...,...,...,...,...
7669861,1,57,1,28.0,0,1,1,51661.0,124.0,109
7669862,1,28,1,50.0,1,0,0,25651.0,152.0,184
7669863,1,47,1,33.0,1,1,0,2630.0,138.0,63
7669864,1,30,1,28.0,0,0,1,38866.0,124.0,119


In [9]:
result_list=[]
for clf in clf_list:
    pred= clf.predict(x)
    result_list.append(pred)
result_list

[array([0, 1, 1, ..., 0, 1, 0]),
 array([0, 1, 1, ..., 0, 1, 0]),
 array([0, 1, 1, ..., 0, 1, 0]),
 array([0, 1, 1, ..., 0, 1, 0]),
 array([0, 1, 1, ..., 0, 1, 0]),
 array([0, 1, 1, ..., 0, 1, 0]),
 array([0, 1, 1, ..., 0, 1, 0])]

In [10]:
pred=[]
for i, id in enumerate(df['id']):
    tmp =0
    for j in result_list:
        tmp+=j[i]
    if tmp>3:
        pred.append(1)
    else:
        pred.append(0)
pred

[0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,


In [11]:
result = pd.DataFrame(df['id'])
result['Response']=pred
result

,id,Response
0,11504798,0
1,11504799,1
2,11504800,1
3,11504801,0
4,11504802,0
...,...,...
7669861,19174659,1
7669862,19174660,0
7669863,19174661,0
7669864,19174662,1


In [12]:
result.to_csv('result.csv',index=False)